# Regresion Logistica

### Importaciones

In [ ]:
import os
import zipfile
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression

### Traemos los datos desde kaggle

In [ ]:
user = userdata.get('KAGGLE_USERNAME')
key = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = user
os.environ["KAGGLE_KEY"] = key
assert user and key, "Faltan los secretos KAGGLE_USERNAME/KAGGLE_KEY"
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
!unzip udea*.zip > /dev/null
!wc *.csv

In [ ]:
train_path = DATA_DIR / "train.csv"
test_path  = DATA_DIR / "test.csv"
sample_path = DATA_DIR / "sample_submission.csv"

df_train = pd.read_csv(train_path)
df_test  = pd.read_csv(test_path)
df_sample = pd.read_csv(sample_path)

df_train(3)

### Preprocesado de la Data

Para el preprocesado usare la funcion propuesta para la entrega #2 con unas pequeñas adaptaciones

In [ ]:
def make_scaler(kind: str):
    kind = (kind or 'standard').lower()
    if kind in ('standard', 'std', 'zscore'):
        return StandardScaler(with_mean=False)  # compatible con matrices dispersas
    else:
        return StandardScaler(with_mean=False)

def build_preprocessor_v2(
    df: pd.DataFrame,
    y_col: str = 'RENDIMIENTO_GLOBAL',
    id_col: str = 'ID',
    num_impute: str = 'median',
    cat_impute: str = 'most_frequent',
    encode_categorical: str = 'onehot',
    ordinal_maps: dict = None,
    scale_numeric: str = 'standard'
):
    data = df.copy()
    y = data.pop(y_col) if y_col in data.columns else None

    # Quitar columnas no predictoras
    for col in [id_col, 'Y_NUMERIC']:
        if col in data.columns:
            data = data.drop(columns=col)

    # Tipos
    num_cols = data.select_dtypes(include=['int64','float64']).columns.tolist()
    cat_cols = data.select_dtypes(include=['object']).columns.tolist()

    # Ordinal vs OneHot (según el mapa recibido)
    ordinal_maps = ordinal_maps or {}
    ordinal_cols = [c for c in cat_cols if c in ordinal_maps]
    onehot_cols  = [c for c in cat_cols if c not in ordinal_maps]

    # Pipelines
    num_pipe = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy=num_impute)),
        ('scaler', make_scaler(scale_numeric))
    ])

    onehot_pipe = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy=cat_impute)),
        ('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist', max_categories=30, sparse=True))
    ])

    ordinal_pipe = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy=cat_impute)),
        ('ordinal', OrdinalEncoder(
            categories=[ordinal_maps[c] for c in ordinal_cols],
            handle_unknown='use_encoded_value', unknown_value=-1
        ))
    ]) if ordinal_cols else 'drop'

    preprocessor = ColumnTransformer(
        transformers=[
            ('num',    num_pipe,    num_cols),
            ('onehot', onehot_pipe, onehot_cols),
            ('ordinal', ordinal_pipe, ordinal_cols)
        ],
        remainder='drop',
        n_jobs=1
    )
    return preprocessor, data, y

In [ ]:
ordinal_maps = {
  'E_VALORMATRICULAUNIVERSIDAD': [
      'NO PAGO MATRICULA',
      'MENOS DE 500 MIL',
      'ENTRE 500 MIL Y MENOS DE 1 MILLON',
      'ENTRE 1 MILLON Y MENOS DE 2.5 MILLONES',
      'ENTRE 2.5 MILLONES Y MENOS DE 4 MILLONES',
      'ENTRE 4 MILLONES Y MENOS DE 5.5 MILLONES',
      'ENTRE 5.5 MILLONES Y MENOS DE 7 MILLONES',
      'MAS DE 7 MILLONES'
  ],
  'E_HORASSEMANATRABAJA': [
      '0',
      'MENOS DE 10 HORAS',
      'ENTRE 11 Y 20 HORAS',
      'ENTRE 21 Y 30 HORAS',
      'MAS DE 30 HORAS'
  ],
  'F_ESTRATOVIVIENDA': [
      'SIN ESTRATO', 'ESTRATO 1', 'ESTRATO 2', 'ESTRATO 3', 'ESTRATO 4', 'ESTRATO 5', 'ESTRATO 6'
  ],
  'F_EDUCACIONPADRE': [
      'NO SABE',
      'NINGUNO',
      'PRIMARIA INCOMPLETA',
      'PRIMARIA COMPLETA',
      'SECUNDARIA (BACHILLERATO) INCOMPLETA',
      'SECUNDARIA (BACHILLERATO) COMPLETA',
      'TECNICA O TECNOLOGICA INCOMPLETA',
      'TECNICA O TECNOLOGICA COMPLETA',
      'EDUCACION PROFESIONAL INCOMPLETA',
      'EDUCACION PROFESIONAL COMPLETA',
      'POSTGRADO'
    ],
    'F_EDUCACIONMADRE': [
      'NO SABE',
      'NINGUNO',
      'PRIMARIA INCOMPLETA',
      'PRIMARIA COMPLETA',
      'SECUNDARIA (BACHILLERATO) INCOMPLETA',
      'SECUNDARIA (BACHILLERATO) COMPLETA',
      'TECNICA O TECNOLOGICA INCOMPLETA',
      'TECNICA O TECNOLOGICA COMPLETA',
      'EDUCACION PROFESIONAL INCOMPLETA',
      'EDUCACION PROFESIONAL COMPLETA',
      'POSTGRADO'
  ]
}

In [ ]:
preproc, X_df, y = build_preprocessor_v2(
    df,
    y_col='RENDIMIENTO_GLOBAL',
    id_col='ID',
    num_impute='median',
    cat_impute='most_frequent',
    encode_categorical='onehot',
    ordinal_maps=ordinal_maps,
    scale_numeric='standard'
)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_df, y, test_size=0.2, stratify=y, random_state=42
)

In [ ]:
logreg = Pipeline(steps=[
    ('prep', preproc),
    ('clf', LogisticRegression(
        multi_class='multinomial',
        solver='saga',
        max_iter=300,
        n_jobs=-1
    ))
])
logreg.fit(X_train, y_train)

In [ ]:
preds = logreg.predict(X_valid)
acc  = accuracy_score(y_valid, preds)
f1m  = f1_score(y_valid, preds, average='macro')

print(f'Accuracy: {acc:.4f} | F1-macro: {f1m:.4f}')
print('\nReporte:\n', classification_report(y_valid, preds))
print('\nMatriz de confusión:\n', confusion_matrix(y_valid, preds, labels=sorted(y.unique())))